In [ ]:
import gensim
import os
import collections
import smart_open
import random

import pandas as pd
import numpy as np


from pprint import pprint
import string
translator = str.maketrans('', '', string.punctuation)

# TODO
- Remove punctuations
- Filtering based on word count

In [ ]:
def get_train_data():
    """
    SQL here 
    """
    data = pd.read_csv('~/Desktop/grocery_products.csv')
    return data

In [ ]:
data = get_train_data()

In [ ]:
print(data.shape)

In [ ]:
selected_data = data[['source_product_id', 'name', 'departments_name', 'categories_name', 'description_en']]
selected_data = selected_data.head(1000)

selected_data

Creating document string

In [ ]:
selected_data['document'] = selected_data['name'] + ' ' + selected_data['departments_name'] + ' ' + selected_data['categories_name'] + ' ' + selected_data['description_en']

In [ ]:
docs = selected_data['document'].tolist()

In [ ]:
selected_data[['document']].iloc[3]

In [ ]:
pp.pprint(docs)
# print(type(docs[0]))

In [ ]:
docs = [d.translate(translator) for d in docs if isinstance(d, str)]

In [ ]:
pprint(docs)

In [ ]:
docs[3]

In [ ]:
def clean_documents(docs):
    # remove common words and tokenize
    stoplist = set('for a of the and to in'.split())
    docs = [[word for word in document.lower().split() if word not in stoplist]
             for document in docs]

    # remove words that appear only once
    from collections import defaultdict
    frequency = defaultdict(int)
    for text in docs:
        for token in text:
            frequency[token] += 2

    docs = [[token for token in text if frequency[token] > 1] for text in docs]

    pprint(docs)
    return docs

In [ ]:
docs = clean_documents(docs)

In [ ]:
dictionary = corpora.Dictionary(docs)
dictionary.save(os.path.join('output_data', 'products_corpus.dict'))

In [ ]:
print(dictionary)

In [ ]:
print(dictionary.token2id)

In [ ]:
corpus = [dictionary.doc2bow(doc) for doc in docs]
corpora.MmCorpus.serialize(os.path.join('output_data', 'products_corpus.mm'), corpus)

In [ ]:
! ls 'output_data'

In [ ]:
DATA_FOLDER = 'output_data'
dictionary = corpora.Dictionary.load(os.path.join(DATA_FOLDER, 'products_corpus.dict'))
corpus = corpora.MmCorpus(os.path.join(DATA_FOLDER, 'products_corpus.mm'))

In [ ]:
print(corpus)

In [284]:
lsi = models.LsiModel(corpus, id2word=dictionary, num_topics=20)
index = similarities.MatrixSimilarity(lsi[corpus])

/Users/vipul/anaconda3/lib/python3.7/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


### Testing a sample document for similarity

In [285]:
sample_doc = 'heathy tea'

In [286]:
vec_bow = dictionary.doc2bow(sample_doc.lower().split())
vec_lsi = lsi[vec_bow]
print(vec_lsi)

[(0, -0.0775871446535606), (1, 0.044932902734822526), (2, 0.313559522202222), (3, -0.08259056778566974), (4, 0.6405941798080496), (5, -0.5738869030231158), (6, 0.15689385276636192), (7, -0.05676383173844955), (8, -0.08302652365167139), (9, 0.03870876085052755), (10, -0.16798086336519183), (11, -0.18689217005885414), (12, -0.06482828541010939), (13, -0.1189417275457018), (14, 0.06099423348934556), (15, 0.02466737461633099), (16, 0.006504616019852107), (17, 0.013228261539041058), (18, 0.0032175476274633754), (19, 0.025148326284397383)]


In [287]:
similarity_scores = index[vec_lsi]
similarity_scores = list(enumerate(similarity_scores))

In [288]:
# Sort in descending order of similarity
sorted_similarity_scores = sorted(similarity_scores, key=lambda tup: tup[1], reverse=True)

In [289]:
print(similarity_scores)

[(0, 0.017232586), (1, 0.006896451), (2, 0.84447944), (3, 0.00938907), (4, -0.015278727), (5, -0.004345529), (6, -0.0027109832), (7, 0.29276645), (8, 0.00035842508), (9, 0.014982313), (10, -0.00016218424), (11, -0.013314433), (12, -0.007321164), (13, 0.025610108), (14, 0.09523198), (15, 0.006657541), (16, -0.003048882), (17, -0.0026737526), (18, 0.00082060695), (19, 0.010752663), (20, -0.0017475784), (21, 0.003514707), (22, -0.0021096244), (23, -0.00893008), (24, 0.00084647536), (25, -0.008126944), (26, -0.00944677), (27, -0.01001901), (28, -0.0047782995), (29, -0.009953357), (30, 0.02085438), (31, 0.010750502), (32, -0.0062630996), (33, -0.00031219423), (34, -0.008293115), (35, -0.004147839), (36, 0.0026223436), (37, -0.0016515031), (38, 0.000121861696), (39, 0.45842776), (40, -0.0027115159), (41, 0.010011874), (42, -0.018043824), (43, 0.027237456), (44, 0.0037400257), (45, -0.0028494075), (46, -0.0012254864), (47, -0.015033498), (48, -0.001776725), (49, 0.00046794116), (50, 0.0064228

In [290]:
print(sorted_similarity_scores)

[(808, 1.0), (646, 0.93641603), (202, 0.90710384), (398, 0.8992729), (717, 0.8940436), (822, 0.893244), (504, 0.8821889), (270, 0.8688731), (853, 0.8588915), (616, 0.8561624), (331, 0.84685045), (2, 0.84447944), (201, 0.84447944), (707, 0.8397509), (405, 0.83736384), (590, 0.8075758), (761, 0.80737066), (296, 0.7826811), (839, 0.7459579), (627, 0.7278456), (522, 0.71641946), (250, 0.7081653), (373, 0.6708495), (389, 0.6136004), (543, 0.6091307), (850, 0.5949092), (100, 0.5785254), (785, 0.56944406), (676, 0.56560767), (588, 0.5481382), (669, 0.509508), (263, 0.5093709), (840, 0.5029664), (184, 0.4850539), (39, 0.45842776), (741, 0.45705935), (128, 0.4492942), (615, 0.4440403), (387, 0.44079012), (851, 0.4376909), (191, 0.37990436), (458, 0.37438044), (256, 0.34444997), (7, 0.29276645), (133, 0.24660951), (127, 0.1742811), (104, 0.14341109), (325, 0.14196733), (629, 0.13116665), (390, 0.12718964), (322, 0.116013706), (605, 0.11238976), (633, 0.096747145), (14, 0.09523198), (791, 0.09331